In [1]:
from easyquery import Query
from astropy.table import Table, Column, vstack

import SAGA
from SAGA import ObjectCuts as C
print(SAGA.__version__)

0.5.8


In [2]:
saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')
saga_database.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr14_nsa{}.fits.gz'

In [3]:
mmt18_hosts = [161174, 52773, 163956, 69028, 144953, 165082, 165707, 145729, 165980, 147606]

In [4]:
saga_targets = SAGA.TargetSelection(saga_database, gmm_parameters='gmm_parameters_no_outlier')

In [5]:
saga_targets.load_object_catalogs(mmt18_hosts, (C.gri_cut & C.fibermag_r_cut & C.is_galaxy & C.is_clean))

In [6]:
score_bins = [150, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
d = np.array([np.searchsorted(base['TARGETING_SCORE'], score_bins) for base in saga_targets.compile_target_list('iter')])

/home/yymao/Downloads/saga/SAGA/targets/assign_targeting_score.py:13: RuntimeWarning: overflow encountered in exp
  return np.where(x > model_parameters[2], np.minimum(np.exp((x-model_parameters[3])*model_parameters[4]), model_parameters[5]), 0.0)


In [13]:
print(SAGA.targets.assign_targeting_score.__doc__)

t = Table(d[:,1:] - d[:,:-1], names=score_bins[:-1])
t.add_column(Column(mmt18_hosts, 'NSAID'), 0)
print("# in each bin")
t.pprint(-1, -1)

print()

t = Table(d[:,1:-1], names=['<{}'.format(s) for s in score_bins[1:-1]])
t.add_column(Column(mmt18_hosts, 'NSAID'), 0)
print("# cumulative")
t.pprint(-1, -1)


    Last updated: 05/07/2018 (for MMT run)
     100 Human selection and Special targets
     150 satellites
     200 within host,  r < 17.77, gri cuts
     300 within host,  r < 20.75, high p_GMM or GMM outliers, gri cuts
     400 within host,  r < 20.75, high-proirity + gri cuts
     500 outwith host, r < 17.77, gri cuts
     600 within host,  r < 20.75, gri cuts, random selection of 50
     700 very high p_GMM, gri cuts
     800 within host,  r < 20.75, gri cuts, everything else
     900 outwith host, r < 20.75, gri cuts
    1000 everything else
    1100 Not in gri/fibermag_r_cut
    1200 Not galaxy
    1300 Not clean
    1400 Has spec but not a satellite
    
# in each bin
NSAID  150 200 300 400 500 600 700 800  900 
------ --- --- --- --- --- --- --- ---- ----
161174   4   1  37 100  12  50  41  218 2763
 52773   2   2  89 150  14  50  28  436 2981
163956   5   5  70 111  13  50  27  493 2730
 69028   4  15 121 248  38  50  36  812 2609
144953   3   6  69 171  24  50  16  566 2592

In [10]:
t = Query('TARGETING_SCORE < 900').filter(saga_targets.object_catalog_cache[0])['TARGETING_SCORE', 'OBJID', 'RA', 'DEC', 'r_mag']
t.rename_column('TARGETING_SCORE', 'SCORE')
t['RA'].format = '%.7f'
t['DEC'].format = '%.7f'
t['r_mag'].format = '%.2f'
t.pprint(-1, -1, )

SCORE        OBJID             RA        DEC     r_mag
----- ------------------- ----------- ---------- -----
  150 1237668298203202132 182.8482068 20.0632228 19.92
  150 1237668298203267092 182.9906665 20.0278487 14.41
  150 1237668298203070473 182.4280443 20.0469188 16.75
  150 1237668298740007188 182.6898152 20.5929283 18.25
  290 1237668298203071372 182.5558721 20.1280558 17.72
  312 1237668298739941498 182.5136409 20.4466898 19.93
  314 1237668298203332906 183.1177589 20.1081726 19.64
  315 1237667915419353805 182.6472958 19.8125902 19.99
  316 1237668298203136683 182.7360724 20.1904022 20.56
  317 1237668298203398405 183.2406742 20.1390866 20.71
  317 1237667915419485104 182.9637545 19.8138805 20.61
  319 1237668298203005569 182.4041371 20.1341974 20.71
  320 1237667915419485058 182.9485653 19.8672097 19.73
  321 1237667915956355578 182.8730349 20.2071582 18.90
  323 1237668298203398459 183.2618908 20.1254050 20.23
  324 1237667915419550104 183.0901849 19.8457099 20.58
  324 1237